In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics 

In [2]:
d = pd.read_csv("Titanic_train.csv")[['Age', 'Fare', 'Pclass', 'Survived', 'Sex', 'SibSp', 'Parch']].dropna()
encoder = {"Sex": {"female": 1, "male": 0}}
d = d.replace(encoder)

# Scaling
def absolute_maximum_scale(series):
    return series / series.abs().max()

for col in d.columns:
    d[col] = absolute_maximum_scale(d[col])


# Constructing the input and target matrices
X = d[['Age', 'Fare', 'Pclass', 'Sex', 'SibSp', 'Parch']]
Y = d['Survived'].values.tolist()


# Split into train and test set
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, shuffle=True)

clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)
            
# make predictions
y_pred_train = clf.predict(X_train)
print("Accuracy on training set: {:.2f}%".format(accuracy_score(y_train, y_pred_train) * 100))
y_pred_test = clf.predict(X_test)
print("Accuracy on test set: {:.2f}%".format(accuracy_score(y_test, y_pred_test) * 100))
print()
print("Feature importance is listed below")

for i in range(6):
    print(X.columns[i],":",clf.feature_importances_[i])

Accuracy on training set: 99.30%
Accuracy on test set: 71.33%

Feature importance is listed below
Age : 0.27485336526961757
Fare : 0.23193922892206995
Pclass : 0.11036475016046873
Sex : 0.3272815397875909
SibSp : 0.04486164306774354
Parch : 0.0106994727925094


In [3]:
test_accuracies = []
train_accuracies = []

for i  in range(100):
    lf = RandomForestClassifier(n_estimators = 100) 

    lf.fit(X_train, y_train)

    y_pred_train = lf.predict(X_train)
    y_pred_test = lf.predict(X_test)

    train_accuracies.append(accuracy_score(y_train, y_pred_train))
    test_accuracies.append(accuracy_score(y_test, y_pred_test))
    
print()
print("Maximum training accuracy is ",max(train_accuracies))
print("Maximum testing accuracy is ",max(test_accuracies))


Maximum training accuracy is  0.9929947460595446
Maximum testing accuracy is  0.7902097902097902


In [4]:
d = pd.read_csv("Titanic_train.csv")[['Age', 'Fare', 'Pclass', 'Survived', 'Sex', 'SibSp', 'Parch','Survived']].dropna()
encoder = {"Sex": {"female": 1, "male": 0}}
d = d.replace(encoder)

# Scaling
def absolute_maximum_scale(series):
    return series / series.abs().max()

for col in d.columns:
    d[col] = absolute_maximum_scale(d[col])


# Making a CSV for Random Forest implementation from scratch
d = d[['Age', 'Fare', 'Pclass', 'Sex', 'SibSp', 'Parch','Survived']]

d.to_csv("train.csv", index=False, header=False)

In [5]:
from random import seed
from random import randrange
from csv import reader
from math import sqrt

def get_split(dataset, n_features):
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    features = list()
    while len(features) < n_features:
        index = randrange(len(dataset[0])-1)
        if index not in features:
            features.append(index)
    for index in features:
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            gini = gini_index(groups, class_values)
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups}

# Load the premade CSV file
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())

def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup

# Basic cross validation
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

# Accuracy finding
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

# Evaluate using cross validation
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores

def test_split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right

# Calculating Gini index
def gini_index(groups, classes):
    # count all samples at split point
    n_instances = float(sum([len(group) for group in groups]))
    # sum weighted Gini index for each group
    gini = 0.0
    for group in groups:
        size = float(len(group))
        # avoid divide by zero
        if size == 0:
            continue
        score = 0.0
        # score the group based on the score for each class
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            score += p * p
        # weight the group score by its relative size
        gini += (1.0 - score) * (size / n_instances)
    return gini


def get_split(dataset, n_features):
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    features = list()
    while len(features) < n_features:
        index = randrange(len(dataset[0])-1)
        if index not in features:
            features.append(index)
    for index in features:
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            gini = gini_index(groups, class_values)
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups}


def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)


def split(node, max_depth, min_size, n_features, depth):
    left, right = node['groups']
    del(node['groups'])
    # check for a no split
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    # check for max depth
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    # process left child
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left, n_features)
        split(node['left'], max_depth, min_size, n_features, depth+1)
    # process right child
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right, n_features)
        split(node['right'], max_depth, min_size, n_features, depth+1)


def build_tree(train, max_depth, min_size, n_features):
    root = get_split(train, n_features)
    split(root, max_depth, min_size, n_features, 1)
    return root


def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']

def subsample(dataset, ratio):
    sample = list()
    n_sample = round(len(dataset) * ratio)
    while len(sample) < n_sample:
        index = randrange(len(dataset))
        sample.append(dataset[index])
    return sample

def bagging_predict(trees, row):
    predictions = [predict(tree, row) for tree in trees]
    return max(set(predictions), key=predictions.count)

def random_forest(train, test, max_depth, min_size, sample_size, n_trees, n_features):
    trees = list()
    for i in range(n_trees):
        sample = subsample(train, sample_size)
        tree = build_tree(sample, max_depth, min_size, n_features)
        trees.append(tree)
    predictions = [bagging_predict(trees, row) for row in test]
    return(predictions)

filename = 'train.csv'
dataset = load_csv(filename)
for i in range(0, len(dataset[0])-1):
    str_column_to_float(dataset, i)
str_column_to_int(dataset, len(dataset[0])-1)
n_folds = 5
max_depth = 10
min_size = 1
sample_size = 1.0
n_features = int(sqrt(len(dataset[0])-1))
for n_trees in [1,3,5,7,9,10]: # Try a few size of trees
    scores = evaluate_algorithm(dataset, random_forest, n_folds, max_depth, min_size, sample_size, n_trees, n_features)
    print("Total number of Trees in this iteration is, ",n_trees)    
    print('The accuracy for this iteration is, ',(sum(scores)/float(len(scores))))
    print()

Total number of Trees in this iteration is,  1
The accuracy for this iteration is,  98.73239436619718

Total number of Trees in this iteration is,  3
The accuracy for this iteration is,  99.43661971830986

Total number of Trees in this iteration is,  5
The accuracy for this iteration is,  100.0

Total number of Trees in this iteration is,  7
The accuracy for this iteration is,  99.71830985915493

Total number of Trees in this iteration is,  9
The accuracy for this iteration is,  99.85915492957747

Total number of Trees in this iteration is,  10
The accuracy for this iteration is,  99.85915492957747



## Observation report

- Age, fare and sex seems to have a high importance in the model.
- On the contrary Parch and Sibsp seem to have very little importance.
- Pclass holds a considerable amount of weight.
- The training accuracy for decision tree as well as random forest comes out to be very similar.
- The testing accuracy for random forest is higher than decision tree by a considerable amount (~5%).
- Maunal implementation of Random Forest seems to be overfitting.